# Obtaining a RCCSD Code using Sympy and Tchau-Spin

<font size=3> $\bullet$ Import Sympy packages

In [1]:
from sympy.physics.secondquant import (AntiSymmetricTensor, wicks,
        F, Fd, NO, evaluate_deltas, substitute_dummies, Commutator,
        simplify_index_permutations, PermutationOperator)
from sympy import (
    symbols, Rational, latex, Dummy
)

# For Sympy (simplification of equations)
pretty_dummies_dict = {
    'above': 'cdefgh',
    'below': 'klmno',
    'general': 'pqrstu'
}

<font size=3> $\bullet$ Define symbols for Sympy

In [2]:
i = symbols('i', below_fermi=True, cls=Dummy)
a = symbols('a', above_fermi=True, cls=Dummy)
j = symbols('j', below_fermi=True, cls=Dummy)
b = symbols('b', above_fermi=True, cls=Dummy)
p, q, r, s = symbols('p,q,r,s', cls=Dummy)

## Build the $\Phi$-normal ordered Hamiltonian

In [3]:
fock = AntiSymmetricTensor('f', (p,), (q,))
pr = NO(Fd(p)*F(q))
V = AntiSymmetricTensor('v',(p,q),(r,s))
pqsr = NO(Fd(p)*Fd(q)*F(s)*F(r))
H = fock*pr + Rational(1,4)*V*pqsr
H

AntiSymmetricTensor(f, (_p,), (_q,))*NO(CreateFermion(_p)*AnnihilateFermion(_q)) - AntiSymmetricTensor(v, (_p, _q), (_r, _s))*NO(CreateFermion(_p)*CreateFermion(_q)*AnnihilateFermion(_r)*AnnihilateFermion(_s))/4

## Build Cluster Operator

In [4]:
def get_T():
    i, j = symbols('i,j', below_fermi=True, cls=Dummy)
    a, b = symbols('a,b', above_fermi=True, cls=Dummy)
    t_ai = AntiSymmetricTensor('t', (a,), (i,))*NO(Fd(a)*F(i))
    t_abij = Rational(1,4)*AntiSymmetricTensor('t', (a,b), (i,j))*NO(Fd(a)*Fd(b)*F(j)*F(i))
    return t_ai + t_abij

get_T()

AntiSymmetricTensor(t, (_a,), (_i,))*NO(CreateFermion(_a)*AnnihilateFermion(_i)) - AntiSymmetricTensor(t, (_a, _b), (_i, _j))*NO(CreateFermion(_a)*CreateFermion(_b)*AnnihilateFermion(_i)*AnnihilateFermion(_j))/4

## Perform the Hausdorff expansion

<font size=3><center> $\bar{H} = e^{-\hat{T}}\hat{H}_Ne^{\hat{T}}$ <br/>
    
 <p>&nbsp;</p>
    
<center>$\bar{H} = \hat{H}_N + [\hat{H}_N, \hat{T}] + \frac{1}{2}[[\hat{H}_N, \hat{T}], \hat{T}] + \frac{1}{3!}[[[\hat{H}_N, \hat{T}], \hat{T}], \hat{T}] + \frac{1}{4!}[[[[\hat{H}_N, \hat{T}], \hat{T}], \hat{T}], \hat{T}]$

In [5]:
C = Commutator
T = get_T()
print("commutator 1...")
comm1 = wicks(C(H, T))
comm1 = evaluate_deltas(comm1)
comm1 = substitute_dummies(comm1)

commutator 1...


In [6]:
T = get_T()
print("commutator 2...")
comm2 = wicks(C(comm1, T))
comm2 = evaluate_deltas(comm2)
comm2 = substitute_dummies(comm2)

commutator 2...


In [7]:
T = get_T()
print("commutator 3...")
comm3 = wicks(C(comm2, T))
comm3 = evaluate_deltas(comm3)
comm3 = substitute_dummies(comm3)

commutator 3...


In [8]:
T = get_T()
print("commutator 4...")
comm4 = wicks(C(comm3, T))
comm4 = evaluate_deltas(comm4)
comm4 = substitute_dummies(comm4)

commutator 4...


## Construct the Similarity Transformed Hamiltonian

In [9]:
eq = H + comm1 + comm2/2 + comm3/6 + comm4/24
eq = eq.expand()
eq = evaluate_deltas(eq)
eq = substitute_dummies(eq, new_indices=True,
        pretty_indices=pretty_dummies_dict)

## Get energy expression

<font size=3><center>$E = \langle | \bar{H} e^{\hat{T}}| \rangle_C$

In [10]:
i, j, k, l = symbols('i,j,k,l', below_fermi=True)
a, b, c, d = symbols('a,b,c,d', above_fermi=True)
energy = wicks(eq, simplify_dummies=True,
        keep_only_fully_contracted=True)
energy

AntiSymmetricTensor(f, (_k,), (_c,))*AntiSymmetricTensor(t, (_c,), (_k,)) - AntiSymmetricTensor(t, (_c,), (_l,))*AntiSymmetricTensor(t, (_d,), (_k,))*AntiSymmetricTensor(v, (_k, _l), (_c, _d))/2 + AntiSymmetricTensor(t, (_c, _d), (_k, _l))*AntiSymmetricTensor(v, (_k, _l), (_c, _d))/4

In [26]:
latex(energy)

'f^{k}_{c} t^{c}_{k} - \\frac{t^{c}_{l} t^{d}_{k} v^{kl}_{cd}}{2} + \\frac{t^{cd}_{kl} v^{kl}_{cd}}{4}'

# Get $T_1$ amplitude equation

<font size=3><center>$E = \langle \Phi_{i}^{a}| \bar{H} e^{\hat{T}}| \rangle_C$

<p>&nbsp;</p>

<center> $E = \langle |i^\dagger a \bar{H} e^{\hat{T}}| \rangle_C$

In [11]:
eqT1 = wicks(NO(Fd(i)*F(a))*eq, simplify_dummies=True, 
             keep_only_fully_contracted=True, simplify_kronecker_deltas=True)
eqT1

-AntiSymmetricTensor(f, (_k,), (_c,))*AntiSymmetricTensor(t, (_c,), (i,))*AntiSymmetricTensor(t, (a,), (_k,)) + AntiSymmetricTensor(f, (_k,), (_c,))*AntiSymmetricTensor(t, (a, _c), (i, _k)) - AntiSymmetricTensor(f, (_k,), (i,))*AntiSymmetricTensor(t, (a,), (_k,)) + AntiSymmetricTensor(f, (a,), (_c,))*AntiSymmetricTensor(t, (_c,), (i,)) + AntiSymmetricTensor(f, (a,), (i,)) - AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(t, (_d,), (i,))*AntiSymmetricTensor(t, (a,), (_l,))*AntiSymmetricTensor(v, (_k, _l), (_c, _d)) - AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(t, (_d,), (i,))*AntiSymmetricTensor(v, (a, _k), (_c, _d)) + AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(t, (a,), (_l,))*AntiSymmetricTensor(v, (_k, _l), (i, _c)) + AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(t, (a, _d), (i, _l))*AntiSymmetricTensor(v, (_k, _l), (_c, _d)) + AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(v, (a, _k), (i, _c)) - AntiSymmetricTensor(t, (_c,), (i

## Get $T_2$ amplitude equation

<font size=3><center>$E = \langle \Phi_{ij}^{ab}| \bar{H} e^{\hat{T}}| \rangle_C$

<p>&nbsp;</p>

<center> $E = \langle |i^\dagger j^\dagger b a \bar{H} e^{\hat{T}}| \rangle_C$

In [12]:
eqT2 = wicks(NO(Fd(i)*Fd(j)*F(b)*F(a))*eq, simplify_dummies=True, 
             keep_only_fully_contracted=True, simplify_kronecker_deltas=True)
eqT2

AntiSymmetricTensor(f, (_k,), (_c,))*AntiSymmetricTensor(t, (_c,), (i,))*AntiSymmetricTensor(t, (a, b), (j, _k)) - AntiSymmetricTensor(f, (_k,), (_c,))*AntiSymmetricTensor(t, (_c,), (j,))*AntiSymmetricTensor(t, (a, b), (i, _k)) + AntiSymmetricTensor(f, (_k,), (_c,))*AntiSymmetricTensor(t, (a,), (_k,))*AntiSymmetricTensor(t, (b, _c), (i, j)) - AntiSymmetricTensor(f, (_k,), (_c,))*AntiSymmetricTensor(t, (b,), (_k,))*AntiSymmetricTensor(t, (a, _c), (i, j)) + AntiSymmetricTensor(f, (_k,), (i,))*AntiSymmetricTensor(t, (a, b), (j, _k)) - AntiSymmetricTensor(f, (_k,), (j,))*AntiSymmetricTensor(t, (a, b), (i, _k)) - AntiSymmetricTensor(f, (a,), (_c,))*AntiSymmetricTensor(t, (b, _c), (i, j)) + AntiSymmetricTensor(f, (b,), (_c,))*AntiSymmetricTensor(t, (a, _c), (i, j)) + AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(t, (_d,), (i,))*AntiSymmetricTensor(t, (a, b), (j, _l))*AntiSymmetricTensor(v, (_k, _l), (_c, _d)) - AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(t, (_d,), (j,

# Using Tchau-Spin

In [13]:
from tchau_spin import *
# For restricted CC
Tensor.rhf = True

<font size=3>$\bullet$ Define indexes

In [14]:
i,j,a,b = Index.new('ijab', 'abab', 'hhpp')
# 'xxxx' indicates that those indexes have any spin
k,l,c,d = Index.new('klcd', 'xxxx', 'hhpp')
index_key = {'i':i,
             'j':j,
             'a':a,
             'b':b,
             'k':k,
             'l':l,
             'c':c,
             'd':d
}

## Processing Energy expression

In [15]:
E = eqfromlatex(latex(energy), index_key)
platex(E)

<IPython.core.display.Math object>

<font size=3>$\bullet$ Simplify Equation

In [16]:
E = E.simplify()
E = E.adapt_space()
platex(E)

<IPython.core.display.Math object>

<font size=3>$\bullet$ Through the Factor object, we can factorize the ERI out for a even simpler expression

In [17]:
E = Factor.factorize_ERI(E)
platex(E)

<IPython.core.display.Math object>

<font size=3>$\bullet$ Process equation. It is necessary to identify the external indexes (in this case none). Also it is optional to provide names.

<font size=3>$\bullet$ Each tensor found in the equation will be given a name. Standard names are a combination of a letter (T, V, f, etc) and o, O, v, V representing the spin and space (hole or particle) of the indexes within the tensor. The auto generated name can be replaced by passing a dictionary as input.

In [18]:
mytensors = {
    'T_OoVv' : 'T2',
    'T_OV' : 'T1',
    'V_OOOO' : 'Voooo',
    'V_OOOV' : 'Vooov',
    'V_OOVV' : 'Voovv',
    'V_OVOV' : 'Vovov',
    'V_OVVV' : 'Vovvv',
    'V_VVVV' : 'Vvvvv',
    'f_OO' : 'fock_OO',
    'f_VV' : 'fock_VV',
    'f_OV' : 'fock_OV'
    
}

pE = process_eq(eq=E, name = 'CC_energy', tensor_labels = mytensors)
pE.write_einsums_out(print_out=True)

CC_energy += 2.0*np.einsum('kc, kc -> ', fock_OV, T1, optimize = 'optimal')
B_OVOV = -1.0*np.einsum('lc, kd -> lckd', T1, T1, optimize = 'optimal')
B_OVOV += -1.0*T2.transpose(0,2,1,3)
B_OVOV += 2.0*T2.transpose(1,2,0,3)
CC_energy += np.einsum('lckd, klcd -> ', B_OVOV, Voovv, optimize = 'optimal')
CC_energy += 2.0*np.einsum('lc, kd, lkcd -> ', T1, T1, Voovv, optimize = 'optimal')



## Processing $T_1$ amplitude equation

In [19]:
T1 = eqfromlatex(latex(eqT1), index_key)
platex(T1)

<IPython.core.display.Math object>

<font size=3>$\bullet$ Simplify

In [20]:
T1 = T1.simplify()
T1 = T1.adapt_space()
T1.sort()
platex(T1)

<IPython.core.display.Math object>

In [21]:
pT1 = process_eq(T1,i,a,name = 'newT1', tensor_labels = mytensors)
pT1.write_einsums_out(print_out=True)

newT1 += fock_OV
newT1 -= np.einsum('ik, ka -> ia', fock_OO, T1, optimize = 'optimal')
newT1 += np.einsum('ca, ic -> ia', fock_VV, T1, optimize = 'optimal')
newT1 -= np.einsum('kc, ic, ka -> ia', fock_OV, T1, T1, optimize = 'optimal')
newT1 += 2.0*np.einsum('kc, ikac -> ia', fock_OV, T2, optimize = 'optimal')
newT1 -= np.einsum('kc, kiac -> ia', fock_OV, T2, optimize = 'optimal')
newT1 -= np.einsum('kc, icka -> ia', T1, Vovov, optimize = 'optimal')
newT1 += 2.0*np.einsum('kc, kica -> ia', T1, Voovv, optimize = 'optimal')
newT1 -= np.einsum('kicd, kadc -> ia', T2, Vovvv, optimize = 'optimal')
newT1 += 2.0*np.einsum('ikcd, kadc -> ia', T2, Vovvv, optimize = 'optimal')
newT1 += -2.0*np.einsum('klac, klic -> ia', T2, Vooov, optimize = 'optimal')
newT1 += np.einsum('lkac, klic -> ia', T2, Vooov, optimize = 'optimal')
newT1 += -2.0*np.einsum('kc, la, lkic -> ia', T1, T1, Vooov, optimize = 'optimal')
newT1 -= np.einsum('kc, id, kadc -> ia', T1, T1, Vovvv, optimize = 'optimal')
newT1 += 2.0*np

## Processing $T_1$ amplitude equation

In [30]:
latex(eqT2)

'f^{k}_{c} t^{c}_{i} t^{ab}_{jk} - f^{k}_{c} t^{c}_{j} t^{ab}_{ik} + f^{k}_{c} t^{a}_{k} t^{bc}_{ij} - f^{k}_{c} t^{b}_{k} t^{ac}_{ij} + f^{k}_{i} t^{ab}_{jk} - f^{k}_{j} t^{ab}_{ik} - f^{a}_{c} t^{bc}_{ij} + f^{b}_{c} t^{ac}_{ij} + t^{c}_{k} t^{d}_{i} t^{ab}_{jl} v^{kl}_{cd} - t^{c}_{k} t^{d}_{j} t^{ab}_{il} v^{kl}_{cd} + t^{c}_{k} t^{a}_{l} t^{bd}_{ij} v^{kl}_{cd} - t^{c}_{k} t^{b}_{l} t^{ad}_{ij} v^{kl}_{cd} - t^{c}_{k} t^{ad}_{ij} v^{bk}_{cd} + t^{c}_{k} t^{ab}_{il} v^{kl}_{jc} - t^{c}_{k} t^{ab}_{jl} v^{kl}_{ic} + t^{c}_{k} t^{bd}_{ij} v^{ak}_{cd} + t^{c}_{i} t^{d}_{j} t^{a}_{k} t^{b}_{l} v^{kl}_{cd} + t^{c}_{i} t^{d}_{j} t^{a}_{k} v^{bk}_{cd} - t^{c}_{i} t^{d}_{j} t^{b}_{k} v^{ak}_{cd} + \\frac{t^{c}_{i} t^{d}_{j} t^{ab}_{kl} v^{kl}_{cd}}{2} + t^{c}_{i} t^{d}_{j} v^{ab}_{cd} - t^{c}_{i} t^{a}_{k} t^{b}_{l} v^{kl}_{jc} - t^{c}_{i} t^{a}_{k} t^{bd}_{jl} v^{kl}_{cd} - t^{c}_{i} t^{a}_{k} v^{bk}_{jc} + t^{c}_{i} t^{b}_{k} v^{ak}_{jc} - t^{c}_{i} t^{b}_{l} t^{ad}_{jk} v^{kl}_{cd} - t^

In [22]:
T2 = eqfromlatex(latex(eqT2), index_key)
platex(T2)

<IPython.core.display.Math object>

<font size=3>$\bullet$ Simplify

In [23]:
T2 = T2.simplify()
T2 = T2.adapt_space()
T2.sort()
platex(T2)

<IPython.core.display.Math object>

<font size=3> $\bullet$ For RHF equations some terms are permutations of another. The object Permutation offers a method to search for such symmetries. Let's look for a permutation $P_+(ij)P_+(ab)$, note that

<center> <font size=3> $P_+(ij)A_{ij} = A_{ij} + A_{ji}$

In [24]:
T2 = Permutation.find_permutations(T2, (i,j), (a,b))
platex(T2)

<IPython.core.display.Math object>

In [25]:
pT2 = process_eq(T2,i,j,a,b, name = 'newT2', tensor_labels = mytensors)
pT2.write_einsums_out(print_out=True)

newT2 += Voovv
newT2 += np.einsum('ic, jd, cdab -> ijab', T1, T1, Vvvvv, optimize = 'optimal')
newT2 += np.einsum('ijcd, cdab -> ijab', T2, Vvvvv, optimize = 'optimal')
newT2 += np.einsum('ka, lb, ijkl -> ijab', T1, T1, Voooo, optimize = 'optimal')
newT2 += np.einsum('klab, ijkl -> ijab', T2, Voooo, optimize = 'optimal')
newT2 -= np.einsum('ic, jd, ka, kbcd -> ijab', T1, T1, T1, Vovvv, optimize = 'optimal')
newT2 -= np.einsum('ic, jd, kb, kadc -> ijab', T1, T1, T1, Vovvv, optimize = 'optimal')
newT2 += np.einsum('ic, ka, lb, lkjc -> ijab', T1, T1, T1, Vooov, optimize = 'optimal')
newT2 += np.einsum('jc, ka, lb, klic -> ijab', T1, T1, T1, Vooov, optimize = 'optimal')
newT2 += np.einsum('klac, ijdb, klcd -> ijab', T2, T2, Voovv, optimize = 'optimal')
newT2 += -2.0*np.einsum('ikac, ljbd, klcd -> ijab', T2, T2, Voovv, optimize = 'optimal')
newT2 += -2.0*np.einsum('lkac, ijdb, klcd -> ijab', T2, T2, Voovv, optimize = 'optimal')
newT2 += np.einsum('kiac, ljdb, lkcd -> ijab', T2, T2, Voovv, o

## The equations above can be coded into a CC program. See the RCCSD.py file for a complete code